In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pprint
pp = pprint.PrettyPrinter(indent=4)

import acctrack.utils.ctd2021_data as data_helper
from acctrack.io.athena_raw_root import AthenaRawRootReader
from acctrack.tools.compare_tracking_algs import TrackAlgComparator
from acctrack.tools.analyze_track_data import TrackAnalyzer


In [8]:
run_number = 242020
event_number = 6800

In [62]:
gnn_path = "/pscratch/sd/x/xju/ITk/ForFinalPaper/run_athena/run/GNN4Pixel"
gnn_reader = AthenaRawRootReader(gnn_path)
gnn_reader.read_file(0)

Tree GNN4ITk is not in /pscratch/sd/x/xju/ITk/ForFinalPaper/run_athena/run/GNN4Pixel/physval.root
Tree GNN4ITk is not in /pscratch/sd/x/xju/ITk/ForFinalPaper/run_athena/run/GNN4Pixel/test.aod.gnnreader.debug.root
/pscratch/sd/x/xju/ITk/ForFinalPaper/run_athena/run/GNN4Pixel contains  1 files and total 1 events.


In [65]:
ckf_path = "/pscratch/sd/x/xju/ITk/ForFinalPaper/run_athena/run/ckfOneEvt"
ckf_reader = AthenaRawRootReader(ckf_path)
ckf_reader.read_file(0)

Tree GNN4ITk is not in /pscratch/sd/x/xju/ITk/ForFinalPaper/run_athena/run/ckfOneEvt/physval.root
Tree GNN4ITk is not in /pscratch/sd/x/xju/ITk/ForFinalPaper/run_athena/run/ckfOneEvt/test.aod.ckf.debug.root
/pscratch/sd/x/xju/ITk/ForFinalPaper/run_athena/run/ckfOneEvt contains  1 files and total 1 events.
Reading file: /pscratch/sd/x/xju/ITk/ForFinalPaper/run_athena/run/ckfOneEvt/Dump_GNN4Itk.root
Number of events: 1 in file /pscratch/sd/x/xju/ITk/ForFinalPaper/run_athena/run/ckfOneEvt/Dump_GNN4Itk.root.


[6800]

In [66]:
gnn_reader.match_to_truth()

AttributeError: 'AthenaRawRootReader' object has no attribute 'match_to_truth'

In [67]:
from acctrack.io import utils_athena_raw_root as utils_raw_root
from acctrack.io import utils_athena_raw as utils_raw_csv

in_file_name = "/pscratch/sd/x/xju/ITk/ForFinalPaper/run_athena/run/GNN4Pixel/Dump_GNN4Itk.root"


In [68]:
import uproot

tree_name = "GNN4ITk"
tree = uproot.open(in_file_name)[tree_name]


In [32]:
all_event_info = tree.arrays(utils_raw_root.all_branches)

In [100]:
detailed_truth = tree.arrays(utils_raw_root.detailed_truth_branch_names + ["nTRK"])[0]

In [101]:
detailed_truth["nTRK"]

1503

In [102]:
my_arrays = [detailed_truth[x].to_numpy() for x in utils_raw_root.detailed_truth_branch_names if "trajectory" not in x]

In [106]:
my_arrays[2][:, :2].shape

(1740, 2)

In [115]:
full_arrays = np.array([my_arrays[0], my_arrays[1], my_arrays[2][:, 0], my_arrays[2][:, 1],
                              my_arrays[3][:, 0], my_arrays[3][:, 1],
                              my_arrays[4][:, 0], my_arrays[4][:, 1]]).T

In [118]:
df_num_matched = pd.DataFrame(full_arrays, columns=["trkid", "num_matched",
                                        "true_pixel_hits", "true_sct_hits",
                                        "reco_pixel_hits", "reco_sct_hits",
                                        "common_pixel_hits", "common_sct_hits"])

In [117]:
full_arrays = np.concatenate([my_arrays[0], my_arrays[1], my_arrays[2][:, :2],
                              my_arrays[3][:, :2], my_arrays[4][:, :2]])
df_num_matched = pd.DataFrame(dict(zip(["trkid", "num_matched",
                                        "true_pixel_hits", "true_sct_hits",
                                        "reco_pixel_hits", "reco_sct_hits",
                                        "common_pixel_hits", "common_sct_hits"],
                                        [my_arrays[0], my_arrays[1], my_arrays[2][:, 0], my_arrays[2][:, 1],
                              my_arrays[3][:, 0], my_arrays[3][:, 1],
                              my_arrays[4][:, 0], my_arrays[4][:, 1]])))

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 2 has 2 dimension(s)

In [76]:
num_matched_arrays = [detailed_truth[x] for x in ["DTTindex", "DTTsize"]]
df_num_matched = pd.DataFrame(dict(zip(["dtt_trkid", "dtt_num_matched"], num_matched_arrays)))

In [83]:
df_num_matched.shape

(1740, 2)

In [73]:
num_matched_arrays

[<Array [[0, 1, 2, 3, 4, ..., 1500, 1501, 1502, 1502]] type='1 * var * int32'>,
 <Array [[3, 1, 1, 1, 1, 1, 1, ..., 1, 1, 1, 1, 1, 1, 2]] type='1 * var * int32'>]

In [33]:
tracks_info = all_event_info[0]

In [35]:
clusters = gnn_reader._read("clusters", event_number)
spacepoints = gnn_reader._read("spacepoints", event_number)

In [39]:
spacepoints.columns

Index(['hit_id', 'x', 'y', 'z', 'cluster_index_1', 'cluster_index_2',
       'SPisOverlap'],
      dtype='object')

In [36]:
pixel_hits = spacepoints[spacepoints["cluster_index_2"] == -1]
strip_hits = spacepoints[spacepoints["cluster_index_2"] != -1]

In [42]:
truth = utils_raw_csv.truth_match_clusters(pixel_hits, strip_hits, clusters)

In [43]:
truth.columns

Index(['hit_id', 'x', 'y', 'z', 'cluster_index_1', 'cluster_index_2',
       'SPisOverlap', 'particle_id'],
      dtype='object')

In [44]:
truth = utils_raw_csv.merge_spacepoints_clusters(truth, clusters)

In [45]:
truth.columns

Index(['hit_id', 'x', 'y', 'z', 'cluster_index_1', 'cluster_index_2',
       'SPisOverlap', 'particle_id', 'hardware', 'cluster_x_1', 'cluster_y_1',
       'cluster_z_1', 'barrel_endcap', 'layer_disk', 'eta_module',
       'phi_module', 'module_id', 'count', 'charge_count', 'loc_eta',
       'loc_phi', 'localDir0', 'localDir1', 'localDir2', 'lengthDir0',
       'lengthDir1', 'lengthDir2', 'glob_eta', 'glob_phi', 'eta_angle',
       'phi_angle', 'norm_x', 'norm_y', 'norm_z', 'subevent', 'barcode',
       'cluster_x_2', 'cluster_y_2', 'cluster_z_2'],
      dtype='object')

In [48]:
truth[["hardware", "barrel_endcap"]].dtypes

hardware         object
barrel_endcap     int32
dtype: object

In [50]:
np.unique(truth.hardware), np.unique(truth.barrel_endcap), np.unique(clusters.barrel_endcap)

(array(['-0.002371282298980759', '-0.0024257369541063746',
        '-0.007478353923206171', ..., '999.4352706555524',
        '999.4634309017697', '999.9315202904162'], dtype=object),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int32),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int32))

In [51]:
cluster_arrays = [tracks_info[x] for x in utils_raw_root.cluster_branch_names
                    if x not in {"CLhardware",
                                "CLparticleLink_barcode",
                                "CLparticleLink_eventIndex"}]

In [52]:
cluster_hardware = np.array(tracks_info["CLhardware"].tolist(), dtype=str)
cluster_columns = [*utils_raw_root.cluster_col_names, "hardware"]
cluster_arrays.append(cluster_hardware)
clusters2 = pd.DataFrame(dict(zip(cluster_columns, cluster_arrays)))

In [55]:
np.unique(clusters2.barrel_endcap)

array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int32)

In [56]:
avallier_filename = "/global/homes/x/xju/m3443/data/GNN4ITK/CHEP2024_data/root_files/user.avallier.mc21_14TeV.601229.PhPy8EG_A14_ttbar_hdamp258p75_SingleLep.DumpGNN4ITk_v5.e8481_s4149_r15238_EXT0/user.avallier.38040858.EXT0._000193.Dump_GNN4Itk.root"
avallier_tree = uproot.open(avallier_filename)[tree_name]
avallier_all_event_info = avallier_tree.arrays(utils_raw_root.all_branches)

In [60]:
barrel_endcap = avallier_all_event_info[0]["CLbarrel_endcap"]

In [61]:
np.unique(barrel_endcap)

<Array [-2, 0, 2] type='3 * int32'>